In [1]:
import os
import random

import torch
from xml.etree import ElementTree as ET
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from pathlib import Path
import glob
import numpy as np
from PIL import Image
import torchvision
import torchvision.models
from tqdm.notebook import tqdm
from torch import nn
import shutil
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patheffects as pe
import torchvision.transforms as transforms
import yaml
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader
from lightning.pytorch.callbacks import LearningRateFinder
from lightning.pytorch import Trainer, LightningModule
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

In [2]:
# Выбор устройства для обучения модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def seed_everything(seed):
    """
        Фисксация всех сидов в программе для корректного
        сравнения оптимизаторов и обучаемых моделей

        :seed: число для фиксации сидов
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed = 42
seed_everything(seed)

In [4]:
import wandb
wandb.login()
wandb.init(project="Emergency_Cars" )

wandb: Currently logged in as: miraellax. Use `wandb login --relogin` to force relogin


In [ ]:
# !yolo task=detect mode=train model=yolov8n.pt data="C:\\Users\\Alex\\PycharmProjects\\Cursach3\\Emergency Vehicles Russia.v3i.yolov8\\data.yaml" epochs=300 imgsz=640 plots=True

In [ ]:
# !yolo task=detect mode=train model=yolov8s.pt data="C:\\Users\\Alex\\PycharmProjects\\Cursach3\\Emergency Vehicles Russia.v3i.yolov8\\data.yaml" epochs=300 imgsz=640 plots=True

In [5]:
# !yolo task=detect mode=train model=yolov5n.pt data="C:\\Users\\Alex\\PycharmProjects\\Cursach3\\Emergency Vehicles Russia.v3i.yolov8\\data.yaml" epochs=300 imgsz=640 plots=True

PRO TIP рџ’Ў Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



  0%|          | 0.00/5.27M [00:00<?, ?B/s]
  3%|2         | 160k/5.27M [00:00<00:03, 1.41MB/s]


New https://pypi.org/project/ultralytics/8.1.25 available рџ�ѓ Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.1 рџљЂ Python-3.9.13 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: task=detect, mode=train, model=yolov5n.pt, data=C:\\Users\\Alex\\PycharmProjects\\Cursach3\\Emergency Vehicles Russia.v3i.yolov8\\data.yaml, epochs=300, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train24, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False


 12%|#1        | 640k/5.27M [00:00<00:01, 3.32MB/s]
 23%|##3       | 1.23M/5.27M [00:00<00:00, 4.65MB/s]
 35%|###4      | 1.83M/5.27M [00:00<00:00, 5.25MB/s]
 48%|####7     | 2.52M/5.27M [00:00<00:00, 5.92MB/s]
 59%|#####9    | 3.12M/5.27M [00:00<00:00, 5.02MB/s]
 71%|#######   | 3.73M/5.27M [00:00<00:00, 5.37MB/s]
 82%|########2 | 4.34M/5.27M [00:00<00:00, 4.31MB/s]
100%|##########| 5.27M/5.27M [00:01<00:00, 5.39MB/s]
wandb: Currently logged in as: miraellax. Use `wandb login --relogin` to force relogin



  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     29184  ultralytics.nn.modules.block.C3              [64, 64, 2]                   
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  3    156928  ultralytics.nn.modules.block.C3              [128, 128, 3]                 
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    296448  ultralytics.nn.modules.block.C3              [256, 256, 1]                 
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1     33024  ultralytics.nn.modules.conv.Conv             [256, 128, 1, 1]              
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [Non

wandb: wandb version 0.16.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in C:\Users\Alex\PycharmProjects\Cursach3\wandb\run-20240310_145322-y9litqpa
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train24
wandb:  View project at https://wandb.ai/miraellax/YOLOv8
wandb:  View run at https://wandb.ai/miraellax/YOLOv8/runs/y9litqpa

train: Scanning C:\Users\Alex\PycharmProjects\Cursach3\Emergency Vehicles Russia.v3i.yolov8\train\labels.cache... 305 images, 0 backgrounds, 0 corrupt: 100%|##########| 305/305 [00:00<?, ?it/s]
train: Scanning C:\Users\Alex\PycharmProjects\Cursach3\Emergency Vehicles Russia.v3i.yolov8\train\labels.cache... 305 images, 0 backgrounds, 0 corrupt: 100%|##########| 305/305 [00:00<?, ?it/s]

val: Scanning C:\Users\Alex\PycharmProjects\Cursach3\Emergency Vehicles Russia.v3i.yolov8\valid\labels.cache... 87 images, 0 backgrounds, 0 corrupt:

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.844      0.812      0.877      0.715

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250       0.87      0.865      0.908      0.756

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.902      0.794      0.883       0.73

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.903      0.806      0.898      0.745

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.853       0.83      0.911      0.747

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.846      0.851      0.899     


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/3 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  33%|###3      | 1/3 [00:00<00:00,  3.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  67%|######6   | 2/3 [00:00<00:00,  3.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|##########| 3/3 [00:00<00:00,  4.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|##########| 3/3 [00:00<00:00,  4.04it/s]

  0%|          | 0/20 [00:00<?, ?it/s]
      9/300      1.97G     0.9994       1.61      1.258         68        640:   0%|          | 0/20 [00:00<?, ?it/s]
      9/300      1.97G     0.9994       1.61      1.258         68        640:   5%|5         | 1/20 [00:00<00:03,  6.33it/s]
      9/300      1.98G     0.9726      1.58


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250       0.89      0.804      0.887      0.731

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.904      0.814      0.901      0.746

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.883      0.862      0.906      0.746

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.897      0.823      0.897      0.728

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.906      0.837      0.904      0.747

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.894      0.833      0.901    


     10/300      1.98G     0.9138      1.564      1.202         62        640:  35%|###5      | 7/20 [00:01<00:01,  6.92it/s]
     10/300      1.98G     0.9208      1.554      1.201         66        640:  35%|###5      | 7/20 [00:01<00:01,  6.92it/s]
     10/300      1.98G     0.9208      1.554      1.201         66        640:  40%|####      | 8/20 [00:01<00:01,  6.61it/s]
     10/300      1.98G     0.9229      1.531      1.196         91        640:  40%|####      | 8/20 [00:01<00:01,  6.61it/s]
     10/300      1.98G     0.9229      1.531      1.196         91        640:  45%|####5     | 9/20 [00:01<00:01,  6.87it/s]
     10/300      1.98G     0.9285      1.526      1.192        105        640:  45%|####5     | 9/20 [00:01<00:01,  6.87it/s]
     10/300      1.98G     0.9285      1.526      1.192        105        640:  50%|#####     | 10/20 [00:01<00:01,  7.09it/s]
     10/300      1.98G     0.9334      1.521      1.186         98        640:  50%|#####     | 10/20 [00:01<00:01, 

In [6]:
# !yolo task=detect mode=train model=yolov5l.pt data="C:\\Users\\Alex\\PycharmProjects\\Cursach3\\Emergency Vehicles Russia.v3i.yolov8\\data.yaml" epochs=300 imgsz=640 plots=True

PRO TIP рџ’Ў Replace 'model=yolov5l.pt' with new 'model=yolov5lu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



New https://pypi.org/project/ultralytics/8.1.25 available рџ�ѓ Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.1 рџљЂ Python-3.9.13 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: task=detect, mode=train, model=yolov5l.pt, data=C:\\Users\\Alex\\PycharmProjects\\Cursach3\\Emergency Vehicles Russia.v3i.yolov8\\data.yaml, epochs=300, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train25, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False

  0%|          | 0.00/102M [00:00<?, ?B/s]
  0%|          | 96.0k/102M [00:00<02:23, 745kB/s]
  0%|          | 176k/102M [00:00<02:25, 733kB/s] 
  0%|          | 480k/102M [00:00<01:02, 1.70MB/s]
  1%|          | 1.00M/102M [00:00<00:34, 3.11MB/s]
  1%|1         | 1.31M/102M [00:00<00:37, 2.82MB/s]
  2%|2         | 2.53M/102M [00:00<00:17, 5.81MB/s]
  3%|3         | 3.14M/102M [00:00<00:17, 5.95MB/s]
  4%|3         | 3.82M/102M [00:00<00:16, 6.27MB/s]
  4%|4         | 4.45M/102M [00:01<00:19, 5.37MB/s]
  5%|5         | 5.25M/102M [00:01<00:16, 6.08MB/s]
  6%|5         | 5.87M/102M [00:01<00:20, 4.96MB/s]
  6%|6         | 6.41M/102M [00:01<00:19, 5.09MB/s]
  7%|6         | 6.94M/102M [00:01<00:22, 4.35MB/s]
  7%|7         | 7.47M/102M [00:01<00:21, 4.61MB/s]
  8%|7         | 7.95M/102M [00:01<00:25, 3.88MB/s]
  8%|8         | 8.36M/102M [00:02<00:27, 3.54MB/s]
  9%|8         | 8.73M/102M [00:02<00:32, 3.05MB/s]


 10                  -1  1    525312  ultralytics.nn.modules.conv.Conv             [1024, 512, 1, 1]             
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  3   2757632  ultralytics.nn.modules.block.C3              [1024, 512, 3, False]         
 14                  -1  1    131584  ultralytics.nn.modules.conv.Conv             [512, 256, 1, 1]              
 15                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 16             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 17                  -1  3    690688  ultralytics.nn.modules.block.C3              [512, 256, 3, False]          
 18                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256


  9%|8         | 9.05M/102M [00:02<00:33, 2.89MB/s]
  9%|9         | 9.34M/102M [00:02<00:34, 2.79MB/s]
  9%|9         | 9.62M/102M [00:02<00:36, 2.67MB/s]
 10%|9         | 9.89M/102M [00:02<00:36, 2.68MB/s]
 10%|9         | 10.2M/102M [00:02<00:36, 2.65MB/s]
 10%|#         | 10.4M/102M [00:02<00:36, 2.66MB/s]
 10%|#         | 10.7M/102M [00:03<00:35, 2.66MB/s]
 11%|#         | 10.9M/102M [00:03<00:36, 2.64MB/s]
 11%|#         | 11.2M/102M [00:03<00:35, 2.65MB/s]
 11%|#1        | 11.5M/102M [00:03<00:35, 2.64MB/s]
 12%|#1        | 11.7M/102M [00:03<00:35, 2.69MB/s]
 12%|#1        | 12.0M/102M [00:03<00:41, 2.27MB/s]
 12%|#2        | 12.3M/102M [00:03<00:36, 2.58MB/s]
 12%|#2        | 12.6M/102M [00:03<00:46, 2.03MB/s]
 13%|#2        | 12.8M/102M [00:04<00:43, 2.17MB/s]
 13%|#2        | 13.1M/102M [00:04<00:49, 1.86MB/s]
 13%|#3        | 13.3M/102M [00:04<01:00, 1.55MB/s]
 13%|#3        | 13.5M/102M [00:04<01:06, 1.40MB/s]
 13%|#3        | 13.6M/102M [00:04<01:09, 1.34MB/s]
 13%|#3    


                   all         87        250       0.91      0.815      0.907      0.751



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.862      0.875      0.918      0.763

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250       0.91      0.842      0.933      0.775

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.885      0.791      0.903       0.73

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.848      0.829      0.876      0.696

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.878      0.837      0.915      0.743

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.841      0.855      0.913   

 53%|#####3    | 54.2M/102M [00:47<00:43, 1.15MB/s]
 53%|#####3    | 54.3M/102M [00:47<00:40, 1.22MB/s]
 53%|#####3    | 54.5M/102M [00:47<00:47, 1.05MB/s]
 54%|#####3    | 54.6M/102M [00:47<00:50, 988kB/s] 
 54%|#####3    | 54.7M/102M [00:48<00:50, 975kB/s]
 54%|#####3    | 54.8M/102M [00:48<00:53, 918kB/s]
 54%|#####3    | 54.9M/102M [00:48<00:54, 905kB/s]
 54%|#####3    | 55.0M/102M [00:48<00:55, 894kB/s]
 54%|#####4    | 55.1M/102M [00:48<00:54, 906kB/s]
 54%|#####4    | 55.2M/102M [00:48<00:55, 890kB/s]
 54%|#####4    | 55.2M/102M [00:48<00:54, 903kB/s]
 54%|#####4    | 55.3M/102M [00:48<00:54, 893kB/s]
 54%|#####4    | 55.4M/102M [00:49<00:54, 888kB/s]
 55%|#####4    | 55.5M/102M [00:49<00:53, 908kB/s]
 55%|#####4    | 55.6M/102M [00:49<00:53, 904kB/s]
 55%|#####4    | 55.7M/102M [00:49<00:56, 858kB/s]
 55%|#####4    | 55.8M/102M [00:49<00:54, 893kB/s]
 55%|#####4    | 56.0M/102M [00:49<00:50, 954kB/s]
 55%|#####5    | 56.0M/102M [00:49<00:50, 962kB/s]
 55%|#####5    | 56.2M/102M

                   all         87        250      0.887      0.868      0.917      0.774

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.838      0.885      0.905       0.76

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.924      0.801      0.905      0.749

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.955      0.828      0.913       0.73

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.944      0.869      0.927      0.763

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
                   all         87        250      0.941      0.826       0.91       0.75

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances    


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  33%|###3      | 1/3 [00:02<00:04,  2.42s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  67%|######6   | 2/3 [00:03<00:01,  1.75s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|##########| 3/3 [00:04<00:00,  1.32s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|##########| 3/3 [00:04<00:00,  1.50s/it]

  0%|          | 0/20 [00:00<?, ?it/s]
      2/300      10.6G      1.027      2.198      1.347         98        640:   0%|          | 0/20 [00:00<?, ?it/s]
      2/300      10.6G      1.027      2.198      1.347         98        640:   5%|5         | 1/20 [00:00<00:18,  1.04it/s]
      2/300      10.6G     0.9486      2.065      1.278         86        640:   5%|5         | 1/20 [00:01<00:18,  1.04it/s]
      2/300      10.6G     0.9486      2.065 

In [7]:
# !yolo task=detect mode=train model=yolov6n.pt data="C:\\Users\\Alex\\PycharmProjects\\Cursach3\\Emergency Vehicles Russia.v3i.yolov8\\data.yaml" epochs=300 imgsz=640 plots=True

Traceback (most recent call last):
  File "C:\Users\Alex\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Alex\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Alex\PycharmProjects\Cursach3\venv\Scripts\yolo.exe\__main__.py", line 7, in <module>
    sys.exit(entrypoint())
  File "C:\Users\Alex\PycharmProjects\Cursach3\venv\lib\site-packages\ultralytics\cfg\__init__.py", line 540, in entrypoint
    model = YOLO(model, task=task)
  File "C:\Users\Alex\PycharmProjects\Cursach3\venv\lib\site-packages\ultralytics\engine\model.py", line 95, in __init__
    self._load(model, task)
  File "C:\Users\Alex\PycharmProjects\Cursach3\venv\lib\site-packages\ultralytics\engine\model.py", line 161, in _load
    self.model, self.ckpt = attempt_load_one_weight(weights)
  File "C:\Users\Alex\PycharmProjects\Cursach3\venv\lib\site-pack

In [6]:
!yolo task=detect mode=train model=yolov9c.pt data="C:\\Users\\Alex\\PycharmProjects\\Cursach3\\Emergency Vehicles Russia.v3i.yolov8\\data.yaml" epochs=300 imgsz=640 plots=True


  0%|          | 0.00/49.4M [00:00<?, ?B/s]
  0%|          | 80.0k/49.4M [00:00<01:51, 465kB/s]
  1%|          | 256k/49.4M [00:00<00:48, 1.06MB/s]
  1%|          | 448k/49.4M [00:00<00:37, 1.39MB/s]
  1%|1         | 600k/49.4M [00:00<00:39, 1.30MB/s]
  1%|1         | 736k/49.4M [00:00<00:58, 880kB/s] 
  2%|1         | 848k/49.4M [00:01<01:37, 523kB/s]
  2%|1         | 928k/49.4M [00:01<02:05, 407kB/s]
  2%|1         | 992k/49.4M [00:01<02:11, 387kB/s]
  2%|2         | 1.02M/49.4M [00:01<02:10, 388kB/s]
  2%|2         | 1.07M/49.4M [00:02<02:17, 369kB/s]
  2%|2         | 1.12M/49.4M [00:02<02:21, 357kB/s]
  2%|2         | 1.16M/49.4M [00:02<02:28, 340kB/s]
  2%|2         | 1.20M/49.4M [00:02<02:24, 351kB/s]
  2%|2         | 1.23M/49.4M [00:02<02:46, 304kB/s]
  3%|2         | 1.27M/49.4M [00:02<03:11, 264kB/s]
  3%|2         | 1.30M/49.4M [00:03<03:36, 233kB/s]
  3%|2         | 1.33M/49.4M [00:03<03:34, 235kB/s]
  3%|2         | 1.36M/49.4M [00:03<03:56, 213kB/s]
  3%|2         | 1.39M